In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = "stabilityai/stable-diffusion-2"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipeline = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipeline = pipeline.to('cuda:0')

/hpi/fs00/home/leon.hermann/mambaforge/envs/ethics/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from src.utils import for_each_prompt
import os

def generate_images(folder:str, setting: str, key: str, obj:str, prefixes: list[str], images_per_prompt: int):
    prompts = []
    for prefix in prefixes:
        prompt = "A photo portrait of a " + prefix + " " + obj + " at " + setting
        prompts.append(prompt)
        
    output = pipeline(prompts, num_images_per_prompt=images_per_prompt)
    for i, prefix in enumerate(prefixes):
        starting_idx = i * images_per_prompt
        for i, image in enumerate(output.images[starting_idx:starting_idx + images_per_prompt]):
            output_folder = f"{folder}/{setting}/{key}/{obj}"
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            image.save(f"{output_folder}/{prefix}_{i}.png")

for_each_prompt("prompts.json", "images", "work", generate_images)
for_each_prompt("prompts.json", "images", "home", generate_images)

  0%|          | 0/50 [00:07<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.22 GiB. GPU 0 has a total capacity of 31.74 GiB of which 327.38 MiB is free. Including non-PyTorch memory, this process has 31.42 GiB memory in use. Of the allocated memory 25.69 GiB is allocated by PyTorch, and 5.35 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)